<h1 style="color:DodgerBlue">Индивидуальный проект</h1>

<h2 style="color:DodgerBlue">Название проекта:</h2>

----

### Вариант задания 10

<h2 style="color:DodgerBlue">Описание проекта:</h2>

----

***Требования к базовому классу Invoice:***

- Атрибуты:
  - Номер фактуры (InvoiceNumber)
  - Дата выдачи (IssueDate)
  - Общая сумма (TotalAmount).

- Методы:
  - CalculateTotal(): метод для расчета общей суммы по фактуре.
  - AddLine(LineItem lineItem): метод для добавления позиции в фактуру.
  - RemoveLine(LineItem lineItem): метод для удаления позиции из фактуры.

***Требования к производным классам:***

- Товарная Фактура (GoodsInvoice): Должна содержать дополнительные
атрибуты, такие как Дата поставки (SupplyDate). Метод AddLine() должен
быть переопределен для добавления информации о дате поставки товара
при добавлении позиции.

- Услуговая Фактура (ServiceInvoice): Должна содержать дополнительные
атрибуты, такие как Дата оказания услуги (ServiceDate).
Метод RemoveLine() должен быть переопределен для добавления
информации о причине аннулирования услуги при удалении позиции.

- Комбинированная Фактура (CombinedInvoice) (если требуется третий класс):
Должна содержать дополнительные атрибуты, такие как Наличие возврата
(ReturnAllowed). Метод CalculateTotal() должен быть переопределен для
учета возможного возврата товара или услуги при расчете общей суммы.

#### Дополнительное задание:
Добавьте к сущестующим классам (базовыму и производным 3-4 атрибута и метода) и реализуйте полиморфизм с перекрытием и прегегрузкой методов, а также generic классы

<h2 style="color:DodgerBlue">Реализация:</h2>

----

In [ ]:
public class LineItem {
  public string Name { get; set; }
  private double _maxPrice;
  private double _minPrice;
  private double _price;
  private int _count;

  private void CheckPrice(double value) {
    if (value < 0) {
      throw new ArgumentOutOfRangeException("Price can't be negative");
    }
  }

  private void CheckCount(int value) {
    if (value < 0) {
      throw new ArgumentOutOfRangeException("Count can't be negative");
    }
  }

  public double Price {
    get {
      return _price;
    }

    set {
      CheckPrice(value);

      _price = value;
      MaxPrice = Math.Max(MaxPrice, _price);
      MinPrice = Math.Min(MinPrice, _price);
    }
  }

  public double MaxPrice {
    get {
      return _maxPrice;
    }

    private set {
      CheckPrice(value);
      _maxPrice = value;
    }
  }

  public double MinPrice {
    get {
      return _minPrice;
    }

    private set {
      CheckPrice(value);
      _minPrice = value;
    }
  }

  public int Count {
    get {
      return _count;
    }

    set {
      CheckCount(value);
      _count = value;
    }
  }

  public LineItem(string name, double price, int count)
  {
    Name = name;
    MaxPrice = price;
    MinPrice = price;
    Price = price;
    Count = count;
  }

  public double GetTotalPrice()
  {
    return Price * Count;
  }
}

public interface ICountItems {
  public int GetCountLineItems();
  public int GetTotalCountItems();
}

public interface IInvoiceSender {
  public void SendInvoiceByEmail(string email);
  public void SendInvoceByMail(string address);
}

public class Invoice : ICountItems, IInvoiceSender
{
  private int _invoiceNumber;
  public DateTime IssueDate { get; set; }
  private DateTime _dueDate;
  public double TotalAmount { get; protected set; }
  public List<LineItem> LineItems { get; protected set; }

  public Invoice(int invoiceNumber, DateTime issueDate, DateTime dueDate)
  {
    InvoiceNumber = invoiceNumber;
    IssueDate = issueDate;
    DueDate = dueDate;
    LineItems = new List<LineItem>();
    TotalAmount = 0;
  }

  public DateTime DueDate {
    get {
      return _dueDate;
    }

    set {
      if (value < IssueDate) {
        throw new ArgumentException("the due date can't be less than the issue date");
      }
      _dueDate = value;
    }
  }

  public int InvoiceNumber {
    get {
      return _invoiceNumber;
    }

    set {
      if (value < 0) {
        throw new ArgumentOutOfRangeException("Invoice number can't be negative");
      }

      _invoiceNumber = value;
    }
  }

  public int GetCountLineItems() {
    return LineItems.Count;
  }

  public int GetTotalCountItems() {
    int total = 0;

    foreach (LineItem item in LineItems) {
      total += item.Count;
    }
    
    return total;
  }

  protected Double CalculateTotal() {
    Double total = 0;

    foreach (LineItem item in LineItems) {
      total += item.GetTotalPrice();
    }

    return total;
  }

  public virtual void AddLine(LineItem lineItem) {
    LineItems.Add(lineItem);
    TotalAmount = CalculateTotal();
    Console.WriteLine($"{lineItem.Name} added successfully");
  }

  public virtual void RemoveLine(LineItem lineItem) {
    if (!LineItems.Contains(lineItem)) {
      throw new ArgumentException($"{lineItem.Name} not found");
    }

    LineItems.Remove(lineItem);
    TotalAmount = CalculateTotal();
    Console.WriteLine($"{lineItem.Name} removed successfully");
  }

  public void SendInvoiceByEmail(string email) {
    Console.WriteLine($"The invoice {InvoiceNumber} was successfully sent by email {email}");
  }
  
  public void SendInvoceByMail(string address) {
    Console.WriteLine($"The invoice {InvoiceNumber} was successfully sent to the address {address}");
  }
}

public class GoodsInvoice : Invoice
{
  public DateTime SupplyDate { get; set; }

  public GoodsInvoice(int invoiceNumber, DateTime issueDate, DateTime supplyDate, DateTime dueDate): base(invoiceNumber, issueDate, dueDate)
  {
    SupplyDate = supplyDate;
  }

  public override void AddLine(LineItem lineItem)
  {
    base.AddLine(lineItem);
    Console.WriteLine($"Supply date: {SupplyDate}.");
  }

  public void ChangePrice(string name, double newPrice) {
    bool exists = false;
    foreach (LineItem item in LineItems) {
      if (item.Name == name) {
        item.Price = newPrice;
        exists = true;
      }
    }

    if (!exists) {
      throw new ArgumentException($"{name} not found");
    }

    TotalAmount = CalculateTotal();
    Console.WriteLine($"The price of {name} has been successfully changed to {newPrice}");
  }
}

public class ServiceInvoice : Invoice
{
  public DateTime ServiceDate { get; set; }

  public ServiceInvoice(int invoiceNumber, DateTime issueDate, DateTime serviceDate, DateTime dueDate): base(invoiceNumber, issueDate, dueDate)
  {
    ServiceDate = serviceDate;
  }

  public override void RemoveLine(LineItem lineItem)
  {
    base.RemoveLine(lineItem);
    Console.WriteLine($"Reason for removal: service no longer needed.");
  }
}

In [ ]:
public class InvoiceCollection<T> where T : Invoice {
  private List<T> _invoices = new List<T>();

  public List<T> Invoices {
    get {
      return _invoices;
    }
  }

  public void AddInvoice(T invoice) {
    if (_invoices.Where(inv => inv.InvoiceNumber == invoice.InvoiceNumber).Any()) {
      throw new ArgumentException("Oops! You can't add invoices with the same numbers.");
    }
    _invoices.Add(invoice);
  }

  public void RemoveInvoice(T invoice) {
    _invoices.Remove(invoice);
  }

  public void RemoveInvoice(int invoiceNumber) {
    _invoices.Remove(_invoices.Single(number => number.InvoiceNumber == invoiceNumber));
  }
}

In [ ]:
// Test new functionality

LineItem phone = new LineItem("Samsung galaxy s25 ultra", 150000, 8);
LineItem phone2 = new LineItem("Samsung galaxy s24 ultra", 100000, 5);
phone.Price = 98000;
phone.Price = 110000;

GoodsInvoice testGoodsInvoice = new GoodsInvoice(1, new DateTime(), new DateTime(), new DateTime());
testGoodsInvoice.AddLine(phone);
testGoodsInvoice.AddLine(phone2);

LineItem testService = new LineItem("Cleaning", 6500, 2);
LineItem testService2 = new LineItem("Walk the pug", 0, 5348);
ServiceInvoice testServiceInvoice = new ServiceInvoice(2, new DateTime(), new DateTime(), new DateTime());
testServiceInvoice.AddLine(testService);
testServiceInvoice.AddLine(testService2);

InvoiceCollection<Invoice> test = new InvoiceCollection<Invoice>();
test.AddInvoice(testGoodsInvoice);
test.AddInvoice(testServiceInvoice);
test.Display();
test.RemoveInvoice(2);
test.RemoveInvoice(testGoodsInvoice);
test.Display();

Samsung galaxy s25 ultra added successfully
Supply date: 1/1/0001 12:00:00 AM.
Samsung galaxy s24 ultra added successfully
Supply date: 1/1/0001 12:00:00 AM.
Cleaning added successfully
Walk the pug added successfully


Submission#3+InvoiceCollection`1[Submission#2+Invoice] Invoices index type value 0 Submission#2+GoodsInvoice Submission#2+GoodsInvoice SupplyDate 0001-01-01 00:00:00Z IssueDate 0001-01-01 00:00:00Z TotalAmount 1380000 LineItems index value 0 Submission#2+LineItem Name Samsung galaxy s25 ultra Price 110000 MaxPrice 150000 MinPrice 98000 Count 8 1 Submission#2+LineItem Name Samsung galaxy s24 ultra Price 100000 MaxPrice 100000 MinPrice 100000 Count 5 DueDate 0001-01-01 00:00:00Z InvoiceNumber 1 1 Submission#2+ServiceInvoice Submission#2+ServiceInvoice ServiceDate 0001-01-01 00:00:00Z IssueDate 0001-01-01 00:00:00Z TotalAmount 13000 LineItems index value 0 Submission#2+LineItem Name Cleaning Price 6500 MaxPrice 6500 MinPrice 6500 Count 2 1 Submission#2+LineItem Name Walk the pug Price 0 MaxPrice 0 MinPrice 0 Count 5348 DueDate 0001-01-01 00:00:00Z InvoiceNumber 2

Invoices,(empty)
